## Step 1 :Prepare data

In [2]:
%pwd
%ls

01_pdf_gold_data.ipynb
01_pdf_loader_llamaindex.ipynb
01_pdf_loader_page-level-summarization.ipynb
01_pdf_retrieval.ipynb
02_video_loader.ipynb
03_video_query.ipynb
__pycache__/
data/


In [ ]:
#Step 0 — Check your current working directory
import os
print(os.getcwd())


/Users/ywxiu/jasp-multimodal-rag/notebooks


In [7]:
print(chunks_path.resolve())
print(chunks_path.exists())


/Users/ywxiu/jasp-multimodal-rag/data/processed/chunks/chunks_Statistical-Analysis-in-JASP-A-guide-for-students-2025.json
True


In [ ]:
#Step 1 – Verify and load your existing JSON
from pathlib import Path
import json

# go one level up from notebooks/
chunks_path = Path("../data/processed/chunks/chunks_Statistical-Analysis-in-JASP-A-guide-for-students-2025.json")

with chunks_path.open("r", encoding="utf-8") as f:
    chunks_data = json.load(f)

len(chunks_data), chunks_data[0].keys()


(556, dict_keys(['text', 'metadata']))

In [ ]:
#quickly inspect one record:
import pprint
pprint.pprint(chunks_data[0])


{'metadata': {'char_length': 532,
              'chunk_id': '1_0_0',
              'images': ['data/page_images/Statistical-Analysis-in-JASP-A-guide-for-students-2025/p001_i02.jpeg'],
              'page': 1,
              'semantic_id': 0,
              'source': 'Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf',
              'split_method': 'semantic+sentence',
              'uuid': 'bf712d2e-c763-4704-9865-08ce0b12d51c'},
 'text': '9    JASP\n'
         '\n'
         'STATISTICAL ANALYSIS IN JASP\n'
         '\n'
         'A GUIDE FOR STUDENTS\n'
         '\n'
         '                            JASP 2025\n'
         '\n'
         'Professor Mark A Goss-Sampson\n'
         'Professor Mark A Goss-Sampson\n'
         '\n'
         '[Image Summaries on this page:]\n'
         '(1) |-----------------|-------------------|------------------|\n'
         '|     -          |    /            |        /\\       |\n'
         '|                  |   /             |              |

In [10]:
#Step 2 – Flatten or extract the fields you’ll need
flattened_chunks = []

for entry in chunks_data:
    meta = entry["metadata"]
    flattened = {
        "chunk_id": meta["chunk_id"],
        "semantic_id": meta["semantic_id"],
        "page": meta["page"],
        "source": meta["source"],
        "split_method": meta["split_method"],
        "char_length": meta["char_length"],
        "text": entry["text"]
    }
    flattened_chunks.append(flattened)

len(flattened_chunks), flattened_chunks[0]


(556,
 {'chunk_id': '1_0_0',
  'semantic_id': 0,
  'page': 1,
  'source': 'Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf',
  'split_method': 'semantic+sentence',
  'char_length': 532,
  'text': '9    JASP\n\nSTATISTICAL ANALYSIS IN JASP\n\nA GUIDE FOR STUDENTS\n\n                            JASP 2025\n\nProfessor Mark A Goss-Sampson\nProfessor Mark A Goss-Sampson\n\n[Image Summaries on this page:]\n(1) |-----------------|-------------------|------------------|\n|     -          |    /            |        /\\       |\n|                  |   /             |              |\n|                -|/               |           \\  |\n|                  |  /              |                |\n|                   | /              |                  |'})

In [13]:
pprint.pprint(flattened_chunks[0])

{'char_length': 532,
 'chunk_id': '1_0_0',
 'page': 1,
 'semantic_id': 0,
 'source': 'Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf',
 'split_method': 'semantic+sentence',
 'text': '9    JASP\n'
         '\n'
         'STATISTICAL ANALYSIS IN JASP\n'
         '\n'
         'A GUIDE FOR STUDENTS\n'
         '\n'
         '                            JASP 2025\n'
         '\n'
         'Professor Mark A Goss-Sampson\n'
         'Professor Mark A Goss-Sampson\n'
         '\n'
         '[Image Summaries on this page:]\n'
         '(1) |-----------------|-------------------|------------------|\n'
         '|     -          |    /            |        /\\       |\n'
         '|                  |   /             |              |\n'
         '|                -|/               |           \\  |\n'
         '|                  |  /              |                |\n'
         '|                   | /              |                  |'}


In [14]:
# Step 4 – Sanity check for duplicates and empty texts
# Check for duplicate chunk IDs
ids = [c["chunk_id"] for c in flattened_chunks]
duplicates = [cid for cid in set(ids) if ids.count(cid) > 1]
print("Duplicate chunk_ids:", duplicates or "None")

# Check for empty texts
empty = [c for c in flattened_chunks if not c["text"].strip()]
print("Empty chunks:", len(empty))


Duplicate chunk_ids: None
Empty chunks: 0


In [ ]:
#optional: save the flattened chunks as a seperated file
out_path = Path("../data/processed/chunks/chunks_flat.json")
with out_path.open("w", encoding="utf-8") as f:
    json.dump(flattened_chunks, f, ensure_ascii=False, indent=2)

print("Saved:", out_path)


## STEP 2: Sampling chunks and using ChatGPT (web or API) to generate realistic user queries per chunk.

#  Define raw TOC mapping (from the PDF Table of Contents)
Provided the complete table of contents as screen shots to chatgpt 5 to generate full sections_raw dictionary with desired naming convention:

“Chapter name: Section name” (when applicable).
For top-level chapters without subsections, the title stands alone.

In [15]:
# --- 1️⃣ Define raw TOC mapping (from the PDF Table of Contents) ---

sections_raw = {
    # --- Front matter ---
    "Preface": 1,

    # --- Core chapters ---
    "Using the JASP Environment": 2,
    "Data Handling in JASP": 8,
    "Data Editing in JASP": 11,
    "JASP Analysis Menu": 12,
    "Descriptive Statistics": 15,
    "Descriptive Statistics: Splitting Data Files": 21,

    # --- Visualization ---
    "Descriptive Data Visualisation": 21,
    "Descriptive Data Visualisation: Basic Plots": 21,
    "Descriptive Data Visualisation: Customisable Plots": 24,
    "Descriptive Data Visualisation: Editing Plots": 28,

    # --- Data quality and transformation ---
    "Exploring Data Integrity": 30,
    "Data Transformation": 38,
    "Effect Size": 42,

    # --- Tests and analyses ---
    "One-sample t-test": 44,
    "Binomial Test": 48,
    "Multinomial Test": 51,
    "Multinomial Test: Chi-square Goodness-of-Fit Test": 53,
    "Multinomial Test: Multinomial and Chi-square Goodness-of-Fit Test": 54,

    # --- Comparing two groups ---
    "Comparing Two Independent Groups": 56,
    "Comparing Two Independent Groups: Independent t-test": 56,
    "Comparing Two Independent Groups: Mann-Whitney U Test": 61,

    "Comparing Two Related Groups": 63,
    "Comparing Two Related Groups: Paired Samples t-test": 63,
    "Comparing Two Related Groups: Wilcoxon’s Signed Rank Test": 67,

    # --- Correlation & Regression ---
    "Correlation Analysis": 69,
    "Regression": 75,
    "Regression: Simple Regression": 78,
    "Regression: Multiple Regression": 81,
    "Regression: Logistic Regression": 88,

    # --- ANOVA families ---
    "Comparing More Than Two Independent Groups": 93,
    "Comparing More Than Two Independent Groups: ANOVA": 93,
    "Comparing More Than Two Independent Groups: Kruskal–Wallis – Non-Parametric ANOVA": 100,

    "Comparing More Than Two Related Groups": 103,
    "Comparing More Than Two Related Groups: RM ANOVA": 103,
    "Comparing More Than Two Related Groups: Friedman’s Repeated Measures ANOVA": 109,

    "Comparing Independent Groups and the Effects of Covariates": 112,
    "Comparing Independent Groups and the Effects of Covariates: ANCOVA": 112,

    "Two-way Independent ANOVA": 120,
    "Two-way Repeated Measures ANOVA": 128,
    "Mixed Factor ANOVA": 137,

    # --- Association & Meta-analysis ---
    "Chi-square Test for Association": 145,
    "Meta-Analysis in JASP": 153,
    "Meta-Analysis in JASP: Effect Size Computation in JASP": 158,
    "Meta-Analysis in JASP: Running the Meta-Analysis in JASP": 159,

    # --- Experimental design section ---
    "Experimental Design and Data Layout in Excel for JASP Import": 164,
    "Experimental Design and Data Layout in Excel for JASP Import: Independent t-test": 164,
    "Experimental Design and Data Layout in Excel for JASP Import: Paired samples t-test": 165,
    "Experimental Design and Data Layout in Excel for JASP Import: Correlation": 166,
    "Experimental Design and Data Layout in Excel for JASP Import: Logistic Regression": 168,
    "Experimental Design and Data Layout in Excel for JASP Import: One-way Independent ANOVA": 169,
    "Experimental Design and Data Layout in Excel for JASP Import: One-way repeated measures ANOVA": 170,
    "Experimental Design and Data Layout in Excel for JASP Import: Two-way Independent ANOVA": 171,
    "Experimental Design and Data Layout in Excel for JASP Import: Two-way Repeated measures ANOVA": 172,
    "Experimental Design and Data Layout in Excel for JASP Import: Two-way Mixed Factor ANOVA": 173,
    "Experimental Design and Data Layout in Excel for JASP Import: Chi-squared - Contingency tables": 174,

    # --- Conceptual & guidance chapters ---
    "Some Concepts in Frequentist Statistics": 175,
    "Which Test Should I Use?": 179,
    "Which Test Should I Use: Comparing one sample to a known or hypothesized population mean": 179,
    "Which Test Should I Use: Testing relationships between two or more variables": 179,
    "Which Test Should I Use: Predicting outcomes": 180,
    "Which Test Should I Use: Testing for differences between two independent groups": 180,
    "Which Test Should I Use: Testing for differences between two related groups": 181,
    "Which Test Should I Use: Testing for differences between three or more independent groups": 181,
    "Which Test Should I Use: Testing for differences between three or more related groups": 182,
    "Which Test Should I Use: Test for interactions between 2 or more independent variables": 182
}


In [16]:
len(sections_raw)

66

create sections_df, which is a Pandas DataFrame — essentially, a table
 — that stores a structured version of the Table of Contents (TOC) with section(chapter or subsection of JASP manual) ,start_page,end_page


In [22]:
# --- Import required libraries ---
import pandas as pd, random

# --- 1️⃣ Apply page offset to match JSON metadata ---
PAGE_OFFSET = 4

# 🧠 Why:
# The page numbers in the Table of Contents (TOC) refer to logical pages in the book,
# but the PDF file used for chunking likely includes front matter (e.g., cover pages, credits),
# causing an offset between actual PDF page indices and TOC numbering.
# Adding +4 ensures that the TOC page numbers align with the metadata
# page values inside your chunk JSON file.

sections = {k: v + PAGE_OFFSET for k, v in sections_raw.items()}

# --- 2️⃣ Convert the section mapping into a DataFrame ---
sections_df = (
    pd.DataFrame([{"section": s, "start_page": p} for s, p in sections.items()])
    .sort_values("start_page")
    .reset_index(drop=True)
)

# --- 3️⃣ Compute end pages automatically ---
# Infer end pages based on the next section
sections_df["end_page"] = sections_df["start_page"].shift(-1) - 1

# Fill NaN for the final section with a small buffer
sections_df.loc[sections_df["end_page"].isna(), "end_page"] = sections_df["start_page"] + 2

# 🔧 Fix cases where the computed end_page < start_page
sections_df.loc[sections_df["end_page"] < sections_df["start_page"], "end_page"] = sections_df["start_page"]

# Ensure integer type for safety
sections_df = sections_df.astype({"end_page": "int"})

print("Total sections:", len(sections_df))
sections_df


Total sections: 66


,section,start_page,end_page
0,Preface,5,5
1,Using the JASP Environment,6,11
2,Data Handling in JASP,12,14
3,Data Editing in JASP,15,15
4,JASP Analysis Menu,16,18
...,...,...,...
61,Which Test Should I Use: Testing for differenc...,184,184
62,Which Test Should I Use: Testing for differenc...,185,185
63,Which Test Should I Use: Testing for differenc...,185,185
64,Which Test Should I Use: Testing for differenc...,186,186


In [34]:
print(list(sections_df))

['section', 'start_page', 'end_page']


In [27]:
import fitz  # PyMuPDF

pdf_path = "../data/raw/Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf"  # ← change to your textbook path

with fitz.open(pdf_path) as doc:
    toc = doc.get_toc(simple=False)  # or simple=True for plain version

if toc:
    print(f"✅ Found {len(toc)} TOC entries:\n")
    for level, title, page, *rest in toc[:30]:  # show first 30 entries
        print(f"Level {level} | Page {page:>3} | {title}")
else:
    print("⚠️ No built-in table of contents found in this PDF.")


⚠️ No built-in table of contents found in this PDF.


In [30]:
import fitz, re, json, os

pdf_path = "../data/raw/Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf"
toc_pages = [3, 4]  # adjust to your manual's TOC pages

toc_map = {}

with fitz.open(pdf_path) as doc:
    for p in toc_pages:
        text = doc[p - 1].get_text("text")
        for line in text.split("\n"):
            m = re.match(r"^(.*?)\s+(\d{1,3})$", line.strip())
            if m:
                title, page = m.groups()
                title = title.strip().rstrip(".")
                page = int(page)
                if title.isupper():
                    toc_map[page] = {"chapter": title}
                else:
                    toc_map[page] = {"section": title}

# ✅ ensure folder exists
os.makedirs("./data/metadata", exist_ok=True)

# save the toc map
with open("./data/metadata/toc_map.json", "w", encoding="utf-8") as f:
    json.dump(toc_map, f, indent=2, ensure_ascii=False)

print(f"✅ Extracted {len(toc_map)} TOC entries and saved to './data/metadata/toc_map.json'")


✅ Extracted 53 TOC entries and saved to './data/metadata/toc_map.json'


In [32]:
import fitz, re, json, os

pdf_path = "../data/raw/Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf"
toc_pages = [3, 4]

toc_map = {}
current_chapter = None
current_section = None

with fitz.open(pdf_path) as doc:
    for p in toc_pages:
        text = doc[p - 1].get_text("text")
        for line in text.split("\n"):
            m = re.match(r"^(\s*)([A-Za-z].*?)\s+(\d{1,3})$", line.strip())
            if not m:
                continue
            indent, title, page = m.groups()
            title = title.strip().rstrip(".")
            page = int(page)

            if title.isupper():  # CHAPTER
                current_chapter = title
                current_section = None
                toc_map[page] = {"chapter": title}
            elif len(indent) > 4:  # SUBSECTION
                toc_map[page] = {"chapter": current_chapter, "section": current_section, "subsection": title}
            else:  # SECTION
                current_section = title
                toc_map[page] = {"chapter": current_chapter, "section": title}

os.makedirs("./data/metadata", exist_ok=True)
with open("./data/metadata/toc_map.json", "w", encoding="utf-8") as f:
    json.dump(toc_map, f, indent=2, ensure_ascii=False)

print(f"✅ TOC map saved with {len(toc_map)} entries.")


✅ TOC map saved with 53 entries.


In [33]:
import json

with open("./data/metadata/toc_map.json") as f:
    toc = json.load(f)

refined = {}
current_chapter = None

for k in sorted(toc.keys(), key=lambda x: float(x)):
    item = toc[k]
    title = item.get("chapter")
    if title.isupper() and not "TEST" in title and not "PLOTS" in title:
        current_chapter = title
        refined[k] = {"chapter": title}
    elif current_chapter:
        refined[k] = {"chapter": current_chapter, "section": title}

with open("./data/metadata/toc_map_refined.json", "w") as f:
    json.dump(refined, f, indent=2)


In [37]:
import json
from pathlib import Path

# path to the enriched file you produced earlier
enriched_path = Path("../data/processed/enriched/Statistical-Analysis-in-JASP-A-guide-for-students-2025_enriched.json")

with open(enriched_path, "r", encoding="utf-8") as f:
    enriched_docs = json.load(f)

len(enriched_docs)


186

In [40]:
toc_pages = [doc for doc in enriched_docs if doc["metadata"]["page"] in (3, 4)]
print(len(toc_pages), "TOC pages found")

print(toc_pages[0]["text"])   # preview
print(toc_pages[1]["text"])   # preview


2 TOC pages found
9    JASP

CONTENTS

PREFACE                                               1
USING THE JASP ENVIRONMENT                            2
DATA HANDLING IN JASP                                 8
DATA EDITING IN JASP                                 11
JASP ANALYSIS MENU                                   12
DESCRIPTIVE STATISTICS                               15
 SPLITTING DATA FILES                                21
DESCRIPTIVE DATA VISUALISATION                       21
 BASIC PLOTS                                         21
 CUSTOMISABLE PLOTS                                  24
 EDITING PLOTS                                       28
EXPLORING DATA INTEGRITY                             30
DATA TRANSFORMATION                                  38
EFFECT SIZE                                          42
ONE-SAMPLE T-TEST                                    44
BINOMIAL TEST                                        48
MULTINOMIAL TEST                                     51
 CHI-SQUA

In [42]:
import re
import pandas as pd

toc_text = "\n".join(t["text"] for t in toc_pages)

# split into lines and match patterns: Title .... PageNumber
pattern = re.compile(r"([A-Za-z’'–\-()&/ ,]+?)\s+(\d{1,3})\b")
matches = pattern.findall(toc_text)

sections = []
for title, page in matches:
    title_clean = " ".join(title.split()).strip(" .")
    page_num = int(page)
    sections.append({"section": title_clean, "start_page": page_num})

# convert to DataFrame
sections_df = pd.DataFrame(sections)
sections_df["end_page"] = sections_df["start_page"].shift(-1, fill_value=sections_df["start_page"].max()+1) - 1
sections_df


,section,start_page,end_page
0,PREFACE,1,1
1,USING THE JASP ENVIRONMENT,2,7
2,DATA HANDLING IN JASP,8,10
3,DATA EDITING IN JASP,11,11
4,JASP ANALYSIS MENU,12,14
...,...,...,...
62,Testing for differences between two related gr...,181,180
63,Testing for differences between three or more ...,181,181
64,Testing for differences between three or more ...,182,1
65,Test for interactions between,2,181


In [68]:
import re
import json
import pandas as pd
from pathlib import Path

# ---------- 1. Load enriched JSON ----------
enriched_path = Path("../data/processed/enriched/Statistical-Analysis-in-JASP-A-guide-for-students-2025_enriched.json")

with open(enriched_path, "r", encoding="utf-8") as f:
    enriched_docs = json.load(f)

print("✅ Loaded pages:", len(enriched_docs))

# ---------- 2. Extract TOC text (pages 3 & 4) ----------
toc_pages = [doc for doc in enriched_docs if doc["metadata"]["page"] in (3, 4)]
toc_text = "\n".join(t["text"] for t in toc_pages)

print("✅ TOC sample:\n", toc_text)


✅ Loaded pages: 186
✅ TOC sample:
 9    JASP

CONTENTS

PREFACE                                               1
USING THE JASP ENVIRONMENT                            2
DATA HANDLING IN JASP                                 8
DATA EDITING IN JASP                                 11
JASP ANALYSIS MENU                                   12
DESCRIPTIVE STATISTICS                               15
 SPLITTING DATA FILES                                21
DESCRIPTIVE DATA VISUALISATION                       21
 BASIC PLOTS                                         21
 CUSTOMISABLE PLOTS                                  24
 EDITING PLOTS                                       28
EXPLORING DATA INTEGRITY                             30
DATA TRANSFORMATION                                  38
EFFECT SIZE                                          42
ONE-SAMPLE T-TEST                                    44
BINOMIAL TEST                                        48
MULTINOMIAL TEST                                

In [70]:
import re
import pandas as pd

lines = [l for l in toc_text.splitlines() if re.search(r"\d{1,3}\b", l)]

# 🧠 Improved pattern:
# - (\d{1,3})$  ensures number is at the END of the line
# - [A-Za-z0-9’'–—“”‘.,&()/%-] in title ensures title can contain words, punctuation, etc.
pattern = re.compile(r"^([A-Za-z0-9’'–—“”‘.,&()/%\- ]*?)\s{2,}(\d{1,3})$")

entries = []
current_chapter = None

for line in lines:
    match = pattern.search(line.strip())
    if not match:
        continue
    title, page = match.groups()
    title = title.strip()
    page = int(page)

    if not line.startswith(" "):  # CHAPTER
        current_chapter = title
        entries.append({
            "chapter": current_chapter,
            "section": None,
            "start_page": page
        })
    else:  # INDENTED = SECTION
        entries.append({
            "chapter": current_chapter,
            "section": title,
            "start_page": page
        })

toc_df = pd.DataFrame(entries)
display(toc_df)


,chapter,section,start_page
0,PREFACE,None,1
1,USING THE JASP ENVIRONMENT,None,2
2,DATA HANDLING IN JASP,None,8
3,DATA EDITING IN JASP,None,11
4,JASP ANALYSIS MENU,None,12
...,...,...,...
59,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between two independen...,180
60,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between two related gr...,181
61,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between three or more ...,181
62,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between three or more ...,182


In [71]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(toc_df)


,chapter,section,start_page
0,PREFACE,None,1
1,USING THE JASP ENVIRONMENT,None,2
2,DATA HANDLING IN JASP,None,8
3,DATA EDITING IN JASP,None,11
4,JASP ANALYSIS MENU,None,12
5,DESCRIPTIVE STATISTICS,None,15
6,DESCRIPTIVE STATISTICS,SPLITTING DATA FILES,21
7,DESCRIPTIVE DATA VISUALISATION,None,21
8,DESCRIPTIVE DATA VISUALISATION,BASIC PLOTS,21
9,DESCRIPTIVE DATA VISUALISATION,CUSTOMISABLE PLOTS,24


In [72]:
# --- Import required libraries ---
import pandas as pd, random

# --- 1️⃣ Apply page offset to match JSON metadata ---
PAGE_OFFSET = 4


toc_df["start_page"] = toc_df["start_page"] + PAGE_OFFSET
toc_df

,chapter,section,start_page
0,PREFACE,None,5
1,USING THE JASP ENVIRONMENT,None,6
2,DATA HANDLING IN JASP,None,12
3,DATA EDITING IN JASP,None,15
4,JASP ANALYSIS MENU,None,16
...,...,...,...
59,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between two independen...,184
60,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between two related gr...,185
61,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between three or more ...,185
62,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between three or more ...,186


In [73]:


# --- 3️⃣ Compute end pages automatically ---
# Infer end pages based on the next section
toc_df["end_page"] = toc_df["start_page"].shift(-1) - 1

# Fill NaN for the final section with a small buffer
toc_df.loc[toc_df["end_page"].isna(), "end_page"] = toc_df["start_page"] + 2

# 🔧 Fix cases where the computed end_page < start_page
toc_df.loc[toc_df["end_page"] < toc_df["start_page"], "end_page"] = toc_df["start_page"]

# Ensure integer type for safety
toc_df = toc_df.astype({"end_page": "int"})

print("Total sections:", len(toc_df))
toc_df


Total sections: 64


,chapter,section,start_page,end_page
0,PREFACE,None,5,5
1,USING THE JASP ENVIRONMENT,None,6,11
2,DATA HANDLING IN JASP,None,12,14
3,DATA EDITING IN JASP,None,15,15
4,JASP ANALYSIS MENU,None,16,18
...,...,...,...,...
59,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between two independen...,184,184
60,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between two related gr...,185,185
61,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between three or more ...,185,185
62,SOME CONCEPTS IN FREQUENTIST STATISTICS,Testing for differences between three or more ...,186,186


In [74]:
import json
from pathlib import Path

# ---------- Load enriched JSON ----------
enriched_path = Path("../data/processed/enriched/Statistical-Analysis-in-JASP-A-guide-for-students-2025_enriched.json")

with open(enriched_path, "r", encoding="utf-8") as f:
    enriched_docs = json.load(f)

print(f"✅ Loaded {len(enriched_docs)} pages")

✅ Loaded 186 pages


In [75]:


# ---------- Helper function ----------
def find_labels(page):
    """Find the chapter and section corresponding to a given page number."""
    row = toc_df[(toc_df["start_page"] <= page) & (toc_df["end_page"] >= page)]
    if not row.empty:
        row = row.iloc[-1]
        return row["chapter"], row["section"]
    # fallback: use the last chapter before this page
    prev = toc_df[toc_df["start_page"] <= page]
    if not prev.empty:
        row = prev.iloc[-1]
        return row["chapter"], row["section"]
    return None, None

# ---------- Assign chapter/section metadata ----------
for doc in enriched_docs:
    page = doc["metadata"].get("page")
    chapter, section = find_labels(page)
    doc["metadata"]["chapter"] = chapter
    doc["metadata"]["section"] = section

# ---------- Verify (sample) ----------
for d in enriched_docs[5:15]:
    print(f"Page {d['metadata']['page']}: {d['metadata']['chapter']} / {d['metadata']['section']}")


Page 6: USING THE JASP ENVIRONMENT / None
Page 7: USING THE JASP ENVIRONMENT / None
Page 8: USING THE JASP ENVIRONMENT / None
Page 9: USING THE JASP ENVIRONMENT / None
Page 10: USING THE JASP ENVIRONMENT / None
Page 11: USING THE JASP ENVIRONMENT / None
Page 12: DATA HANDLING IN JASP / None
Page 13: DATA HANDLING IN JASP / None
Page 14: DATA HANDLING IN JASP / None
Page 15: DATA EDITING IN JASP / None


In [76]:
# Save to a new file
out_path = enriched_path.parent / (enriched_path.stem + "_with_chapters_sections.json")

with open(out_path, "w", encoding="utf-8") as f:
    json.dump(enriched_docs, f, indent=2, ensure_ascii=False)

print(f"✅ Saved enriched file with chapter & section metadata:\n{out_path}")


✅ Saved enriched file with chapter & section metadata:
../data/processed/enriched/Statistical-Analysis-in-JASP-A-guide-for-students-2025_enriched_with_chapters_sections.json


In [79]:
import json
from pathlib import Path

# Path to your enriched file with chapter/section info
out_path = Path("../data/processed/enriched/Statistical-Analysis-in-JASP-A-guide-for-students-2025_enriched_with_chapters_sections.json")

# Load and preview
with open(out_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Show the last 3 blocks in a readable format
for d in data[-3:]:
    print(json.dumps(d, indent=2, ensure_ascii=False))
    print("-" * 120)


{
  "id_": "842cfde0-595b-42e6-b2f9-5be48a4389a1",
  "embedding": null,
  "metadata": {
    "source": "Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf",
    "page": 184,
    "images": [],
    "chapter": "SOME CONCEPTS IN FREQUENTIST STATISTICS",
    "section": "Testing for differences between two independent groups"
  },
  "excluded_embed_metadata_keys": [],
  "excluded_llm_metadata_keys": [],
  "relationships": {},
  "metadata_template": "{key}: {value}",
  "metadata_separator": "\n",
  "text_resource": {
    "embeddings": null,
    "text": "9    JASP\n\n    Predicting outcomes\n\n    Data type\n\n    Continuous    Ordinal    Nominal\n\n   More than one\npredictor variable?\n\n    No    Yes\n\n  Simple       Multiple    Ordinal    Logistic\nregression    regression    regression    regression\n                                          tables\nCurrently not available in\n           JASP\n    Testing for differences between two independent groups\n\n    Data type\n\n    Conti